# Analyzing nitrate data from xylem sap
- Uses calibration curve info from previous step
- Uses rigorous error propagation to calculate error
- Evaluates xylem sap data collected as fruit set

In [2]:
import pandas as pd
import numpy as np
import pickle
from scipy.stats import t

In [3]:
#Re-open the curve to use to analyze our samples
with open('../02-calibration_curve/calibration_curve.pkl','rb') as f:
    cal_curve=pickle.load(f)

In [4]:
#Re-define this function so we can use it again
def uncertainty(df, cal, sample_col='sample_id',rf_col='rf'):
    """
    Uses the uncertainty in a calibration curve measurement found in Harris'
    Quantitative Chemical Analysis Eighth Edition (Equation 4-27)
    
    Takes as input our calibration curve (saved as a pickle) and a dataframe
    with multiplicate (e.g. triplicate) measurements to give a single
    mean value and its uncertainty

    Assumes dataframe already has a calculated response factor column,
    could probably be generalized to do this as well but seems unnecessary
    """
    m=cal['slope']
    a=cal['intercept']
    s_y=cal['s_yx']
    x_bar=cal['x_mean']
    Sxx=cal['Sxx']
    n_cal=cal['n']
    y_bar=a+m*x_bar
    t_val=t.ppf(0.975,n_cal-2) #Two-sided 95% confidence interval

    results=[]
    for sid, grp in df.groupby(sample_col):
        y_u=grp[rf_col].to_numpy()
        k=len(y_u)
        y_u_bar=np.mean(y_u)
        x_hat=(y_u_bar-a)/m
        s_x=(s_y/m)*np.sqrt((1/k)+(1/n_cal)+((y_u_bar-y_bar)**2)/(m**2*Sxx)) #Equation 4-27
        ci_lo=x_hat-t_val*s_x
        ci_hi=x_hat+t_val*s_x
        results.append({
            "sample":sid,
            "n_rep":k,
            "rf_mean":y_u_bar,
            "conc_mean":x_hat,
            "s_x":s_x,
            "CI_low":ci_lo,
            "CI_high":ci_hi
        })
    return pd.DataFrame(results)

In [5]:
#Read nitrate data
fs_data=pd.read_csv("../01-input_data/fruit_set.csv")

In [6]:
fs_data.head(5)

,sample,rep,no3_area,istd_area,analysis_day,n_add
0,FS-O-1,1,2.22,10.16,1,0.0
1,FS-O-1,2,2.09,10.71,1,0.0
2,FS-O-1,3,2.24,10.71,1,0.0
3,FS-O-2,1,1.27,13.22,1,0.0
4,FS-O-2,2,1.68,15.58,1,0.0


In [7]:
#Let's add response factors
fs_data['rf']=fs_data['no3_area']/fs_data['istd_area']

In [8]:
fs_data.head(5)

,sample,rep,no3_area,istd_area,analysis_day,n_add,rf
0,FS-O-1,1,2.22,10.16,1,0.0,0.218504
1,FS-O-1,2,2.09,10.71,1,0.0,0.195145
2,FS-O-1,3,2.24,10.71,1,0.0,0.209150
3,FS-O-2,1,1.27,13.22,1,0.0,0.096067
4,FS-O-2,2,1.68,15.58,1,0.0,0.107831


In [9]:
#Let's summarize analytical reps since they're less important for what we're looking at
#We'll need to keep n_add info for downstream analysis
fs_summary=uncertainty(fs_data,cal_curve,sample_col="sample",rf_col="rf")
fs_summary.head(5)

,sample,n_rep,rf_mean,conc_mean,s_x,CI_low,CI_high
0,FS-15N-1,3,1.894745,0.429943,0.032616,0.362302,0.497584
1,FS-15N-2,3,1.496906,0.338468,0.032737,0.270575,0.406361
2,FS-15N-3,3,3.035858,0.692320,0.032417,0.625091,0.759548
3,FS-15N-4,3,8.154848,1.869330,0.034238,1.798324,1.940336
4,FS-15N-5,3,3.852337,0.880053,0.032412,0.812834,0.947272


In [10]:
#Merge metadata
fs_summary_merged=fs_summary.merge(fs_data[['sample','n_add']].drop_duplicates(),on='sample',how='left')

In [11]:
fs_summary_merged.head(5)

,sample,n_rep,rf_mean,conc_mean,s_x,CI_low,CI_high,n_add
0,FS-15N-1,3,1.894745,0.429943,0.032616,0.362302,0.497584,3.75
1,FS-15N-2,3,1.496906,0.338468,0.032737,0.270575,0.406361,3.75
2,FS-15N-3,3,3.035858,0.692320,0.032417,0.625091,0.759548,3.75
3,FS-15N-4,3,8.154848,1.869330,0.034238,1.798324,1.940336,3.75
4,FS-15N-5,3,3.852337,0.880053,0.032412,0.812834,0.947272,3.75


In [12]:
fs_summary_merged.to_csv("fs_summary.csv",index=False)